In [32]:
import os 

import imageio.v3 as iio
import torch

In [9]:
img_arr = iio.imread('../data/p1ch4/image_dog/pexels-muhannad-alatawi-58997.jpg')
img_arr.shape, type(img_arr)

((1365, 2048, 3), numpy.ndarray)

In [15]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
out.shape


torch.Size([3, 1365, 2048])

In [20]:
img.transpose(2, 0).shape, img.shape # see the problem here?

(torch.Size([3, 2048, 1365]), torch.Size([1365, 2048, 3]))

In [21]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [30]:
# how I did it
for i in range(batch_size):
    img = iio.imread(f'../data/p1ch4/cat-images/cat{i+1}.png')
    img = torch.from_numpy(img)
    img = img.permute(2, 0, 1)

    batch[i] = img


In [35]:
data_dir = '../data/p1ch4/cat-images/'
filenames = [name for name in os.listdir(data_dir) 
            if os.path.splitext(name)[-1] == '.png']

for i, filename in enumerate(filenames):
    img = iio.imread(os.path.join(data_dir, filename))
    img = torch.from_numpy(img)
    img = img.permute(2, 0, 1)
    
    img = img[:3]
    batch[i] = img


In [37]:
batch = batch.to(dtype=torch.float32)
batch /= 255.

In [40]:
n_channels = batch.shape[1]

for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

In [45]:
dir_path = '../data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083/'
vol_arr = iio.imread(dir_path, plugin='DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%25/99 files (25.3%61/99 files (61.6%94/99 files (94.9%99/99 files (100.0%)
  Found 1 correct series.


(512, 512)

In [52]:
vol = torch.from_numpy(vol_arr)
vol = vol.unsqueeze(dim=0)
vol.shape

torch.Size([1, 512, 512])

In [62]:
import imageio
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [63]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)
 
vol.shape

torch.Size([1, 99, 512, 512])